In [1]:
import sys
sys.path.insert(0, "../pydlshogi2/")

import argparse
import logging

import torch
import torch.optim as optim
from dataloader import HcpeDataLoader
from network.policy_value_resnet import PolicyValueNetwork

In [2]:
class args:
    train_data = "../../input/train.hcpe"
    test_data = "../../input/test.hcpe"
    gpu = 0
    epoch = 1
    batchsize = b = 1
    testbatchsize = 512
    lr = 1e-3
    checkpoint = "checkpoints/checkpoint-{epoch:03}.pth"
    resume = False
    eval_interval = 100
    log = '../../input/log.txt'

In [3]:
# utils

def accuracy(y, t):
    """方策の正解率"""
    return (torch.max(y, 1)[1] == t).sum().item() / len(t)

def binary_accuracy(y, t):
    """価値の正解率"""
    pred = y >= 0
    truth = t >= 0.5
    return pred.eq(truth).sum().item() / len(t)

def save_checkpoint():
    """チェックポイント保存"""
    path = args.checkpoint.format(**{"epoch": epoch, "step": t})
    logging.info("Saving the checkpoint to {}".format(path))
    checkpoint = {
        "epoch": epoch,
        "t": t,
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, path)

In [4]:
if __name__ == '__main__':
    
    device = torch.device(f"cuda:{args.gpu}")

    # モデル
    model = PolicyValueNetwork()
    model.to(device)

    # オプティマイザ
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.0001)

    # 損失関数
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    bce_with_logits_loss = torch.nn.BCEWithLogitsLoss()

    # Dataloader
    train_dataloader = HcpeDataLoader(args.train_data, args.batchsize, device, shuffle=True)
    test_dataloader = HcpeDataLoader(args.test_data, args.testbatchsize, device)
    
    epoch = t = 0

## check dataloader

In [5]:
x, move_label, result = train_dataloader.sample()

In [6]:
x.size(), move_label, result

(torch.Size([1, 104, 9, 9]),
 tensor([839], device='cuda:0'),
 tensor([[0.]], device='cuda:0'))

In [7]:
y1, y2 = model(x)

In [8]:
y1.size(), move_label, y2, result

(torch.Size([1, 2187]),
 tensor([839], device='cuda:0'),
 tensor([[-0.2050]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.]], device='cuda:0'))

In [9]:
y1, move_label

(tensor([[ 0.3082,  0.1503,  0.8668,  ...,  1.0607,  0.8628, -1.3890]],
        device='cuda:0', grad_fn=<AddBackward0>),
 tensor([839], device='cuda:0'))

In [10]:
results = []
for _ in range(100):
    _, _, result = train_dataloader.sample()
    results.append(float(result.cpu().numpy()))

In [11]:
results

[1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.5,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0]

In [12]:
torch.onnx.export(model, x, "../../output/network.onnx")